In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from IPython.display import display

In [ ]:
df = pd.read_csv("data.csv")
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


# Подготовка и анализ данных

Произведем предобработку данных:
1. Удалим ненужные столбцы
2. Преобразуем тип данных столбца date_time к datetime, чтобы произвести разбиение на обучающую, валидационную и тестовую выборки
3. Выведем максимальную и минимальную даты, чтобы понять, с каким периодом времени мы работаем
4. Отсортируем датасет по дате/времени и разобъем данные на train, val и test сеты. Test - показы рекламы в последний день, val - в предпоследний, train - все остальные.

In [ ]:
df = df.drop(["banner_id0", "banner_id1", "rate0", "rate1", "g0",
              "g1", "coeff_sum0", "coeff_sum1", "impressions"], axis=1)

df["date_time"] = pd.to_datetime(df.date_time)
df = df.sort_values("date_time")
min_date = df["date_time"].iloc[0].date()
max_date = df["date_time"].iloc[-1].date()
print("Min date: {}, Max date: {}".format(min_date, max_date))
df.drop(index=df.iloc[:1, :].index.tolist(), inplace=True)
min_date = df["date_time"].iloc[0].date()
print("Real Min date: {}".format(min_date))

df_test = df[(df['date_time'].dt.date == max_date)]
df = df[(df['date_time'].dt.date < max_date)]

Min date: 2021-09-01, Max date: 2021-10-02
Real Min date: 2021-09-26


Обратим внимание на лишнее наблюдение - самое первое. На всякий случай удалим его.

Ниже разделяем тренировочное и валидационное множества.

In [ ]:
from datetime import datetime


val_date = datetime.strptime('2021-10-01', "%Y-%m-%d").date()
df_val = df[(df['date_time'].dt.date == val_date)]
df = df[(df['date_time'].dt.date < val_date)]
df_val

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks
2747021,2021-10-01 00:00:00,49,1239,5334084916871444044,0,2,0,0
20675,2021-10-01 00:00:00,17,104,8043427820305743649,3,3,1,1
7465011,2021-10-01 00:00:00,14,0,7558598582596132463,0,0,0,0
12361989,2021-10-01 00:00:00,9,49,8247166999584885670,0,0,0,0
6811089,2021-10-01 00:00:00,168,49,5703620665238317782,0,0,0,0
...,...,...,...,...,...,...,...,...
3594598,2021-10-01 23:59:59,254,584,284430210636077854,0,2,10,0
8604214,2021-10-01 23:59:59,34,47,1801023712706150066,1,2,5,0
1633515,2021-10-01 23:59:59,3,1239,2731395125248566348,0,2,0,0
6571049,2021-10-01 23:59:59,139,49,7391999856880902949,0,0,0,0


В первом задании данные уже были проанализированы, поэтому не будем повторяться.

# Feature engineering

1. Количество предшествующих показов той же кампании, большее 23, объединим в отдельную категорию.
2. Из даты и времени показа вытащим час и воспользуемся им при крафтинге новой фичи.
3. Сконструируем следующие интеракции, то есть создадим новые признаки показа из существующих путем их объединения:
- ОС, место на сайте - на какой платформе и в каком месте сайта (зоне) был показан баннер.
- Страна, время - время показа рекламы юзеру в таймзоне юзера, то есть в какой стране и в какой час был показ.
4. Уберем лишние столбцы
5. Изменим типы данных предикторов


In [ ]:
def feature_engineering(df):
    df.loc[df.campaign_clicks > 23, "campaign_clicks"] = 24     # было "много" показов той же кампании

    df["oaid_hash"] = df["oaid_hash"]
    df['hour'] = df["date_time"].dt.hour
    df["local_user_time"] = (df['country_id'].astype('str') + '_' + df['hour'].astype('str'))
    df['os_zone'] = (df['os_id'].astype('str') + '_' + df['zone_id'].astype('str'))

    df = df.drop(["date_time", "zone_id", "hour"], axis=1)
    df = df.astype("category")

    return df


df = feature_engineering(df)

In [ ]:
df.shape

(12049045, 8)

In [ ]:
df.head()

,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,local_user_time,os_zone
5041415,29,1834033519797437404,1,3,0,0,0_0,3_41
1442602,188,7416450538971744701,2,2,15,0,15_0,2_1
7232498,52,1832228443297591417,2,2,5,0,5_0,2_17
14938691,73,4180077124914749282,1,4,13,0,13_0,4_47
11536774,266,1459689388363839798,1,0,1,0,1_0,0_48


Произведем label encoding наших скрафченных фичей, чтобы из, например, 3_41 получить уникальный числовой идентификатор категории.

Фитим энкодер на тренировочных данных и энкодим их:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder


categories_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
categories_encoder.fit(df[["local_user_time", "os_zone"]])
df[["local_user_time", "os_zone"]] = categories_encoder.transform(df[["local_user_time", "os_zone"]])
df[["local_user_time", "os_zone"]] = df[["local_user_time", "os_zone"]].astype("int").astype("category")
df

,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,local_user_time,os_zone
5041415,29,1834033519797437404,1,3,0,0,0,6323
1442602,188,7416450538971744701,2,2,15,0,144,3970
7232498,52,1832228443297591417,2,2,5,0,288,4502
14938691,73,4180077124914749282,1,4,13,0,96,7894
11536774,266,1459689388363839798,1,0,1,0,192,1648
...,...,...,...,...,...,...,...,...
13129571,145,2812994566358629873,0,1,7,0,352,2435
11159519,161,8481381842118306791,0,0,1,0,208,558
4748458,161,6502523589229511323,1,0,1,0,208,1754
2161751,1026,4446800550048584676,1,4,11,0,64,8192


Создадим маленькую функцию для трансформации, чтобы не повторять один и тот же код для трансформации val и test множеств. И заэнкодим их тоже.

In [ ]:
def transform_categories(categories_encoder, df):
    df[["local_user_time", "os_zone"]] = categories_encoder.transform(df[["local_user_time", "os_zone"]])
    df[["local_user_time", "os_zone"]] = df[["local_user_time", "os_zone"]].astype("int").astype("category")
    return df

In [ ]:
df_val = feature_engineering(df_val)
df_val = transform_categories(categories_encoder, df_val)
df_val

,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,local_user_time,os_zone
2747021,1239,5334084916871444044,0,2,0,0,0,5424
20675,104,8043427820305743649,3,3,1,1,192,6071
7465011,0,7558598582596132463,0,0,0,0,0,319
12361989,49,8247166999584885670,0,0,0,0,0,2033
6811089,49,5703620665238317782,0,0,0,0,0,558
...,...,...,...,...,...,...,...,...
3594598,584,284430210636077854,0,2,10,0,40,4968
8604214,47,1801023712706150066,1,2,5,0,304,5268
1633515,1239,2731395125248566348,0,2,0,0,16,5135
6571049,49,7391999856880902949,0,0,0,0,16,311


In [ ]:
df_test = feature_engineering(df_test)
df_test = transform_categories(categories_encoder, df_test)
df_test

,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,local_user_time,os_zone
14196412,1239,6628179337000354250,0,1,0,0,0,2435
8706638,174,2436793977145729294,0,3,0,0,0,6390
13000378,175,6622864732614000542,0,2,9,0,384,4300
9767447,76,3615498569651227068,0,1,3,0,240,2125
9054327,428,684235863208116380,0,1,10,0,24,3038
...,...,...,...,...,...,...,...,...
10139863,180,1030486855470422958,0,2,6,0,328,4904
5597133,92,5392241310084555538,0,1,0,0,16,3745
8660907,1235,5569517219693927594,0,4,0,0,16,7089
2991997,2,4888551521096100763,0,0,0,0,16,1


# Преобразование данных

Для того чтобы мы могли обучить FFM из библиотеки xlearn, нам нужно преобразовать данные в libffm формат. Для этого воспользуемся уже написанной для этой задачи функцией, только немного ее откорректируем, убрав лишнее.

Функция для преобразования датафрейма в формат libffm взята отсюда: https://github.com/wngaw/blog/blob/master/xlearn_example/src/utils.py

In [ ]:
import math
from tqdm import tqdm



def _convert_to_ffm(path, df, type, target, categories, encoder):
    # Flagging categorical and numerical fields
    print('convert_to_ffm - START')
    for x in categories:
        if(x not in encoder['catdict']):
            print(f'UPDATING CATDICT: categorical field - {x}')
            encoder['catdict'][x] = 1

    nrows = df.shape[0]
    with open(path + str(type) + "_ffm.txt", "w") as text_file:

        # Looping over rows to convert each row to libffm format
        for n, r in tqdm(enumerate(range(nrows)), "Formatting", nrows):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow[target]))  # Set Target Variable here

            # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(encoder['catdict'].keys()):

                # For a new field appearing in a training example
                if(x not in encoder['catcodes']):
                    encoder['catcodes'][x] = {}
                    encoder['currentcode'] += 1
                    encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                # For already encoded fields
                elif(datarow[x] not in encoder['catcodes'][x]):
                    encoder['currentcode'] += 1
                    encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                code = encoder['catcodes'][x][datarow[x]]
                datastring = datastring + " "+str(i)+":" + str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

    return encoder

Трансформируем все 3 сета и записываем в отдельные файлы.

In [ ]:
cols = df.columns.tolist()
ind = cols.index("clicks")
cols.pop(ind)
cols

['banner_id',
 'oaid_hash',
 'campaign_clicks',
 'os_id',
 'country_id',
 'local_user_time',
 'os_zone']

In [ ]:
encoder = {"currentcode": 0,  # Unique index for each numerical field or categorical variables
           "catdict": {},  # Dictionary that stores numerical and categorical variables
           "catcodes": {}}

encoder = _convert_to_ffm("", df, "train", "clicks", cols, encoder)

In [ ]:
encoder = _convert_to_ffm("", df_val, "val", "clicks", cols, encoder)

convert_to_ffm - START


Formatting: 100%|██████████| 1643448/1643448 [08:46<00:00, 3123.67it/s]


In [ ]:
encoder = _convert_to_ffm("", df_test, "test", "clicks", cols, encoder)

convert_to_ffm - START


Formatting: 100%|██████████| 2128978/2128978 [11:30<00:00, 3082.84it/s]


# Обучение модели

Обучать будем модель FFM

## Подбор гиперпараметров

Для того чтобы подобрать наиболее хорошие гиперпараметры, воспользуемся поиском по решетке, а качество будем проверять на валидационной выборке (предпоследний день исходных данных).

Подибрать будем следующие гиперпараметры:
- Размерность эмбеддингов. Даже при k = 8 последняя итерация грид серча упала, поэтому большие размерности пытаться обучать большого смысла нет.
- Learning rate.
- Параметр регуляризации lambda
- Количество эпох константное, потому что xlearn по дефолту производит early stopping, когда для модели указано валидационное множество.

In [1]:
import xlearn as xl
from sklearn.model_selection import ParameterGrid

In [ ]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("train_ffm.txt")
ffm_model.setValidate("val_ffm.txt")
params_grid = {"task": ["binary"],
               "k": [4, 8],
               "lr": [0.1, 0.01],
               "lambda": [0.001, 0.1],
               "metric": ["auc"],
               "epoch": [30]}

for param in ParameterGrid(params_grid):
    print("Fitting with parameters: {}".format(param))
    ffm_model = xl.create_ffm()
    ffm_model.setTrain("train_ffm.txt")
    ffm_model.setValidate("val_ffm.txt")
    ffm_model.fit(param, "model.out")
    print("Fitting finished!\n\n")

Fitting with parameters: {'epoch': 30, 'k': 4, 'lambda': 0.001, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 4, 'lambda': 0.001, 'lr': 0.01, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 4, 'lambda': 0.1, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 4, 'lambda': 0.1, 'lr': 0.01, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 8, 'lambda': 0.001, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 8, 'lambda': 0.001, 'lr': 0.01, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 8, 'lambda': 0.1, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
Fitting finished!


Fitting with parameters: {'epoch': 30, 'k': 8, 'lambda': 0.1, 'lr': 0.01, 'metric': 'auc', 't

xlearn к сожалению (или к счастью) показывает свой громозкий вывод не в ноутбуке, а в консоли. Результаты обучения в процессе оптимизации гиперпараметров на валидации можно найти в файле log_gridsearch.txt. Отметим, что на последней итерации кернел крашнулся, и последний набор гиперпараметров проверить не удалось.



Лучшими гиперпараметрами на валидации оказались следующие: {'epoch': 30, 'k': 4, 'lambda': 0.001, 'lr': 0.1, 'task': 'binary'}, а Log-loss на валидации такой: 0.146529

Теперь обучим модель с этими гиперпараметрами, запустим инференс на тестовом множестве и оценим качество на тесте.

In [2]:
param = {'epoch': 30, 'k': 4, 'lambda': 0.001, 'lr': 0.1, 'task': 'binary'}

ffm_model = xl.create_ffm()
ffm_model.setTrain("train_ffm.txt")
ffm_model.fit(param, "best_model.out")
ffm_model.setTest("test_ffm.txt")
ffm_model.setSigmoid()
ffm_model.predict("best_model.out", "predictions.txt")

Log-loss на тесте составил 0.1352, отчет можно найти в файле log_inference.txt.

В первом задании при использовании линейной модели, обученной на придуманных мной интеракциях, я получил log-loss =  0.1348, то есть совсем немного лучше, чем сейчас у FFM. На мой взгляд, причины у этого следующие:
1. Линейная модель получилась действительно хорошой благодаря хорошо подобранным интеракциям, и потому ее скор побить не так просто. Кроме того, было подобрано оптимальное значение единственного гиперпараметра - регуляризации. Но при этом не будем забывать, что логистическая регрессия обучалась намного дольше, хотя и не крашилась от количества данных в отличие от FFM.
2. У FFM есть несколько гиперпараметров, а значит нужно испытать много комбинаций значений гиперпараметров, чтобы найти оптимальный набор. Скорее всего, если продолжить тюнить модель и добавить в гридсерч больше разных значений, то можно получить модель, способную побить сильную линейную модель из первого ДЗ, но это не точно, потому что уже при размерности эмбеддинга = 8 произошел краш, а лучшее качество может быть как раз достижимо при большой размерности эмбеддингов. То есть тут мы утыкаемся в вычислительные ресурсы.
3. Возможно для FFM стоило подобрать либо побольше интеракций, либо сделать их больше (чтобы входили 3 переменные, как в большинстве интеракций в первом дз, а не две, но это могло бы быть избыточно).


Таким образом, обученная модель FFM на тесте оказалась чуть хуже логистической регрессии из первого ДЗ (на 0.0004), но это объясняется тем, что линейная модель вышла хорошей, и чтобы ее побить, FFM нужно просто дольше тюнить, перебирая больше комбинаций параметров и имея достаточные вычислительные ресурсы.